In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 16, 10

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from torch import nn,optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics.functional import f1
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split

import joblib

In [2]:
path_input = '../input/headlines/summary.csv'
df = pd.read_csv(path_input)
df['summary'] = df['summary'].replace(r'\n', '', regex = True)
df.head()

,text,summary
0,و هو يعتبر متحف جديد العهد ، يقدم مجموعة من ال...,المتحف العربي للفن الحديث
1,يعتبر فيلاجيو مول من أهم المجمعات التجارية و أ...,فيلاجيو مول
2,هو المنطقة البحرية الممتدة على مجموعة كيلومترا...,كورنيش الدوحة
3,يعتبر هذا المنتجع من أجمل الأماكن الساحلية الم...,منتجع شاطئ سيلين
4,و هو منتجع مطل على البحر ، و يحوي العديد من أم...,منتجع الغارية


In [3]:
df = df[:100_000]
len(df)

100000

In [4]:
train, val = train_test_split(df, test_size=0.1, random_state=42)

train.to_csv("train.csv",index=False)
val.to_csv("val.csv",index=False)

In [5]:
# tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

# text_token_counts = train['text'].apply(lambda x : len(tokenizer.encode(x)))
# summary_token_counts = train['summary'].apply(lambda x : len(tokenizer.encode(x)))

# fig, (ax1, ax2) = plt.subplots(1, 2)
# sns.boxplot(text_token_counts, ax=ax1)
# ax1.set_title('full text token counts')
# sns.boxplot(summary_token_counts, ax=ax2)
# ax2.set_title('summary text token counts')

In [6]:
class SummaryDataset(Dataset):
    def __init__(
        self,
        data,
        text_max_token_len,
        summary_max_token_len
    ):
        self.tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
        self.data = data
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        text = data_row['text']

        text_encoding = self.tokenizer(
            text,
            max_length=self.text_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        summary_encoding = self.tokenizer(
            data_row['summary'],
            max_length=self.summary_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )
        
        labels = summary_encoding['input_ids']
        labels[labels == self.tokenizer.pad_token_id] = -100
        
        return dict(
            input_ids=text_encoding['input_ids'].flatten(),
            attention_mask=text_encoding['attention_mask'].flatten(),
            labels=labels.flatten(),
            decoder_attention_mask=summary_encoding['attention_mask'].flatten()
        )

In [7]:
class SummaryDataModule(pl.LightningDataModule):
    def __init__(self,train_path,val_path,batch_size=8, text_max_token_len = 400, summary_max_token_len = 17):
        super().__init__()
        self.train_path,self.val_path= train_path,val_path
        self.batch_size = batch_size
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len
    
    def setup(self,stage=None):
        train = pd.read_csv(self.train_path)
        val = pd.read_csv(self.val_path)
        self.train_dataset = SummaryDataset(data=train,
                                            text_max_token_len=self.text_max_token_len,
                                            summary_max_token_len=self.summary_max_token_len)
        self.val_dataset = SummaryDataset(data=val,
                                          text_max_token_len=self.text_max_token_len,
                                          summary_max_token_len=self.summary_max_token_len)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)

In [8]:
class T5SumModel(pl.LightningModule):
    def __init__(self, lr=0.0001):
        super().__init__()
        self.lr = lr
        self.model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
    
    def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):
        output = self.model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels,
            decoder_attention_mask=decoder_attention_mask
        )
        return output.loss, output.logits
    
    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=self.lr)
    
    def training_step(self, batch, batch_size):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        decoder_attention_mask = batch['decoder_attention_mask']

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_size):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        decoder_attention_mask = batch['decoder_attention_mask']

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

In [9]:
dm = SummaryDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=1)

# First Time only
trained_model = T5SumModel() 

# Load checkpoint
trained_model = T5SumModel.load_from_checkpoint(
    '../input/fine-t5-arabicsummarization/lightning_logs/version_0/checkpoints/epoch=0-step=89999.ckpt'
)

# Traing
# trainer = pl.Trainer(gpus=-1,
#                      max_epochs=1,
#                      default_root_dir='.',
#                      callbacks=[EarlyStopping(monitor="val_loss")])
# trainer.fit(trained_model,dm)

Downloading:   0%|          | 0.00/730 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.17G [00:00<?, ?B/s]

In [10]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

trained_model.freeze()

def summarizeText(text, mymodel):
    text_encoding = tokenizer(
        text,
        max_length=1000,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    generated_ids = mymodel.generate(
        input_ids=text_encoding['input_ids'],
        attention_mask=text_encoding['attention_mask'],
        max_length=150,
        num_beams=2,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for gen_id in generated_ids
    ]
    return "".join(preds)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [11]:
test = val.sample(10)
for i in range(len(test)):
    sample_row = test.iloc[i]
    text = sample_row['text']
    print('Text : ', text)
    print('\n')
    print('Summary (Ground truth) : ',sample_row['summary'])
    print('\n')
    summary_without_tuning = summarizeText(text, model)
    print("Summary before tuning", summary_without_tuning)
    print('\n')
    summary_with_tuning = summarizeText(text, trained_model.model)
    print("Summary after tuning", summary_with_tuning)
    print('\n')
    print('-------------------------------------------------------------------------------------------------')

Text :  من أركان الإيمان الإيمان بالله و رسله و منهم سيدنا محمد صلى الله عليه و سلم ؛ فقد أرسله الله للإنس و الجن يحمل رسالة الهداية ، و المؤمن يحمل هذا الاعتقاد في قلبه ، و يظهر في سلوكه ، فللنبي صلى الله عليه و سلم على أمته حقوق في طاعته ، و اتباعه ، و احترام مقامه ، و توقيره بينها الله في كتابه و النبي صلى الله عليه و سلم في سنته ، و فيما يأتي سنبين هذه الحقوق مع أدلتها . 



Summary (Ground truth) :  نبوة محمد صلى الله عليه و سلم


Summary before tuning في هذا الوقت من كل عام، يؤدي الإيمان بالله و رسله إلى توحيد الأمن والنبي صلى الله عليه و سلم.


Summary after tuning الإيمان بالله


-------------------------------------------------------------------------------------------------
Text :  تعد هذه المرحلة هي المرحلة النهائية لفصل مشتقات النفط عن بعضها البعض ، و تتم باستخدام العمليات الفيزيائية ، أو العمليات الكيميائية ، مثل : استخدام محلول الصودا الكاوية ؛ لتنقية البوتاجاز و البنزين من كبريتيد الهيدروجين ، أو استخدام و حمض الكبريتيك المركز لتنقية الكيروسين من المواد الكبريتية و العطر